Kiwi Budget : A Database of New Zealand Economic Activities (1996 ~ 2022)
Authors : Ali, Jerry, Jeffrey

Abstract

This database contains information from various major social economical activities of New Zealand as well as its key monetary policy indicators. All data are time series based, they have been collected from websites like “interest.co.nz”, “interest.co.nz”, “interest.co.nz” and so on. The reuse potential includes finance department of New Zelanad companies for training machine learning algorithms that does the forecasting as well as visulisation.  This database is available through xxx??

KeyWords : Monetary policy, Economic activities, 

(2)Methods
Step2 
1: Sourcing, Tidying and Enhancing New Zealand Economy Dataset

We have installed following packages ("RSelenium", "Writexl" and "here" packages) utilized
following libraries 

1) magrittr
The magrittr (to be pronounced with a sophisticated french accent) package has two aims: decrease development time and improve readability and maintainability of code. Or even shortr: make your code smokin’ (puff puff)!
2) polite
Be responsible when scraping data from websites by following polite principles: introduce yourself, ask for permission, take slowly and never ask twice. 
3) rvest
Wrappers around the 'xml2' and 'httr' packages to make it easy to download, then manipulate, HTML and XML
4) here
The goal of the here package is to enable easy file referencing in project-oriented workflows. In contrast to using setwd(), which is fragile and dependent on the way you organize your files, here uses the top-level directory of a project to easily build paths to files.
5) RSelenium
Provides a set of R bindings for the 'Selenium 2.0 WebDriver' (see <https://www.selenium.dev/documentation/> for more information) using the 'JsonWireProtocol' (see <https://github.com/SeleniumHQ/selenium/wiki/JsonWireProtocol> for more information). 'Selenium 2.0 WebDriver' allows driving a web browser natively as a user would either locally or on a remote machine using the Selenium server it marks a leap forward in terms of web browser automation. Selenium automates web browsers (commonly referred to as browsers). Using RSelenium you can automate browsers locally or remotely. 

In [2]:
install.packages(c("RSelenium", "writexl", "here"))

Installing packages into ‘/home/jeffreychi/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



In [3]:
# CPI data
library(magrittr)
library(polite)
library(rvest)
library(here)
library(RSelenium)

here() starts at /home/jeffreychi/Downloads/Group Project



Setting up Browser environment that scrapping infoshare.stats.govt.nz website
#todo, Error in curl::curl_fetch_disk

In [4]:
data_dir = paste(here(), "data", sep = "/")

eCaps <- list(
  chromeOptions = 
    list(prefs = list(
        "profile.default_content_settings.popups" = 0L,
        "download.prompt_for_download" = FALSE,
        "directory_upgrade" = TRUE,
        "download.default_directory" = data_dir),
      args = list(
        "--disable-blink-features=AutomationControlled"
      )),
      useAutomationExtension = FALSE,
      excludeSwitches = list(
        "enable-automation"
      )
)

rD <- rsDriver(verbose = FALSE, browser = "chrome", chromever = "106.0.5249.61",
                extraCapabilities = eCaps)
remDr <- rD$client
remDr$maxWindowSize()
remDr$navigate("https://infoshare.stats.govt.nz/infoshare/Default.aspx")

ERROR: Error in curl::curl_fetch_disk(url, x$path, handle = handle): Unrecognized content encoding type. libcurl understands deflate, gzip content encodings.


In [ ]:
Define utility function for retrieving html contents

In [5]:
# Wrapper of remoteDriver$findElement, enable wait and check for 3 times
# rmd:  remote driver client
# p_using: `using` param of findElement
# p_value: `value` param of findElement
get_tree_element <- function(rmd, p_using, p_value) {
    sleep_secs = 2
    vis_check = 0

    # wait for progress valid
    Sys.sleep(sleep_secs)
    while (sleep_secs < 10 & vis_check < 30) {
        vis = remDr$executeScript("return document.getElementById(arguments[0]).style.visibility",
                            args = list("ctl00_MainContent_goProgress", ""))
        # wait for progress finish
        if (vis == "visible") {
            # in progress, keep wait
            vis_check = vis_check + 1
            sleep_secs = 2
            Sys.sleep(sleep_secs)
            next
        } else {
            # wait for element load
           Sys.sleep(sleep_secs) 
        }
        
        try(suppressMessages(rmd$findElement(using = p_using, value = p_value)), silent = TRUE)
        if (rmd$status == 0) {
            elem = rmd$findElement(using = p_using, value = p_value)
            return(elem)
        } else if (remDr$status == 7) {
            # wait for 2 seconds
            Sys.sleep(sleep_secs)
        }
        sleep_secs = sleep_secs * 2
    }
    return(NULL)
}

get_element <- function(rmd, p_using, p_value) {
    sleep_secs = 2

    # wait for progress valid
    Sys.sleep(sleep_secs)
    while (sleep_secs < 10) {
        try(suppressMessages(rmd$findElement(using = p_using, value = p_value)), silent = TRUE)
        if (rmd$status == 0) {
            elem = rmd$findElement(using = p_using, value = p_value)
            return(elem)
        } else if (remDr$status == 7) {
            # wait for 2 seconds
            Sys.sleep(sleep_secs)
        }
        sleep_secs = sleep_secs * 2
    }
    return(NULL)
}

Navigate into predefined hyperlink and retrieve contents

In [6]:
elem_lv_one = get_tree_element(remDr, "link text", "Work income and spending")
elem_lv_one$clickElement()

# wait for loading
elem_lv_two = get_tree_element(remDr, "link text",  "Household Labour Force Survey - HLF")
elem_lv_two$clickElement()

elem_lv_three = get_tree_element(remDr, "link text", "Labour Force Status for people aged 15 to 64 years: Seasonally Adjusted (Qrtly-Mar/Jun/Sep/Dec)")
elem_lv_three$clickElement()

ERROR: Error in get_tree_element(remDr, "link text", "Work income and spending"): object 'remDr' not found


Define constant variables for css selectors

In [7]:
# setup options of labour data for extract data
season_adj  = get_element(remDr, "css selector", "span#ctl00_MainContent_ctl02_lblSelectAll")
time_range  = get_element(remDr, "css selector", "span#ctl00_MainContent_ctl07_lblSelectAll")
obs_range  = get_element(remDr, "css selector", "span#ctl00_MainContent_ctl04_lblSelectAll")
submit  = get_element(remDr, "css selector", "input#ctl00_MainContent_btnGo")

season_adj$clickElement()
time_range$clickElement()
obs_range$clickElement()
submit$clickElement()
# get footnotes for finishing page loading
foot = get_element(remDr, "link text", "Footnotes")

page_source = remDr$getPageSource()[[1]]

Warning message in get_element(remDr, "css selector", "span#ctl00_MainContent_ctl02_lblSelectAll"):
“restarting interrupted promise evaluation”


ERROR: Error in get_element(remDr, "css selector", "span#ctl00_MainContent_ctl02_lblSelectAll"): object 'remDr' not found


Invoke read_html method for scrapping html contents that fetches 

In [8]:
# parse the page source
doc = xml2::read_html(page_source)
labour_table = doc %>% html_table()

ERROR: Error in xml2::read_html(page_source): object 'page_source' not found


After sucessfully retrieve the contents from the website, we are proceed the wrangling stage

Data Wrangling & Save

We are using R library "visdat" for visualization

In [ ]:
library(visdat)
library("writexl")

# labour data
labour <- labour_table[[8]]
labour[2,1] = "date"
column_name = labour[2,]
colnames(labour) = column_name
labour <- labour[-c(1,2),]
labour %>% head(10)

vis_dat(labour)

Now we proceed to the data wrangling stage

Introduction

All data are time-series data; the maximum granularity of time is quarterly, the smaller granularity is monthly, and occasionally there are several points in an individual month; we use the average value as the current month's data so that the minimum granularity can be considered as monthly.

The data will be uniformly converted into quarterly time scales:

    When data are transformed, quarterly data for non-rate of change will simply be treated as an arithmetic average of monthly data, such as the unemployment rate data.
    𝑢𝑛𝑒𝑚𝑝𝑙𝑜𝑦𝑚𝑒𝑛𝑡_𝑟𝑎𝑡𝑒_𝑄1=𝑢𝑛𝑒𝑚𝑝𝑙𝑜𝑦𝑚𝑒𝑛𝑡_𝑟𝑎𝑡𝑒_𝐽𝑎𝑛+𝑢𝑛𝑒𝑚𝑝𝑙𝑜𝑦𝑚𝑒𝑛𝑡_𝑟𝑎𝑡𝑒_𝐹𝑒𝑏+𝑢𝑛𝑒𝑚𝑝𝑙𝑜𝑦𝑚𝑒𝑛𝑡_𝑟𝑎𝑡𝑒_𝑀𝑎𝑟3

When the data are converted, the quarterly data for the rate of change will simply be treated as the product of the months, as is the case with CPI data.
𝐶𝑃𝐼_𝑄1=((1+𝐶𝑃𝐼_𝐽𝑎𝑛)×(1+𝐶𝑃𝐼_𝐹𝑒𝑏)×(1+𝐶𝑃𝐼_𝑀𝑎𝑟))−1

Data

Each data contains different start time, we first find a common time start point of each data, this time point is January 1999 is the start time point of OCR data, and the end of duration is September, 2022. This notebook will wrangling the following data:

    HPI
    CPI
    OCR
    Exchange rate
    Unemployment rate



We have utilized the following packages into the data wrangling process
1) library(timeDate) The 'timeDate' class offers not only date and time functionality but it also offers sophisticated calendar manipulations for business days, weekends, public and ecclesiastical holidays.
2) library(tidyverse) The 'tidyverse' is a set of packages that work in harmony because they share common data representations and 'API' design.
3) library(visdat) The visdat package provides visualisations of an entire dataframe at once. Initially inspired by csv-fingerprint, visdat provides tools to create heatmap-like visualisations of an entire dataframe. visdat provides 2 main functions: vis_dat and vis_miss.
4) library(here) The goal of the here package is to enable easy file referencing in project-oriented workflows.
5) library(readxl) The readxl package makes it easy to get data out of Excel and into R. 

In [ ]:
# Normalization for each time formats
# install.packages("timeDate")
library(timeDate)
library(tidyverse)
library(visdat)
library(here)
library(readxl)


Define common function that does the conversion from data source tiem formate to unified time formate

In [ ]:

# convert "1999Q1" to "1999-03"
Yq2Ym <- function(date) {
           dt = str_split(date, "Q")
           y = dt[[1]][[1]]
           q = strtoi(dt[[1]][[2]], 10)
           m = q * 3
           mm = str_pad(m, 2, side = "left", pad = 0)
           ym = paste(y, mm, sep = "-")
           ym
       }

# convert "1999-3-29" to "1999-03"
Yqd2Ym <- function(date) {
           dt = str_split(date, "-")
           y = dt[[1]][[1]]
           m = strtoi(dt[[1]][[2]], 10)
           mm = str_pad(m, 2, side = "left", pad = 0)
           ym = paste(y, mm, sep = "-")
           ym
       }

# Convert "1999Q1" to the last day of quarter
Yq2Date <- function(date) {
           ym = Yq2Ym(date)
           ymd = paste(ym, "01", sep = "-")
           dt = as.Date(timeLastDayInMonth(ymd))
           dt
       }


# Convert "1999-03-29" to the last day of month
Yqd2Date <- function(date) {
          ym = Yqd2Ym(date)
          ymd = paste(ym, "01", sep = "-")
          dt = as.Date(timeLastDayInMonth(ymd))
          dt
       }

# convert "1999-03" to "1999Q1"
Ym2Yq <- function(date) {
           dt = str_split(date, "-")
           y = dt[[1]][[1]]
           m = strtoi(dt[[1]][[2]], 10)
           q = (m - 1) %/% 3 + 1
           yq = paste(y, q, sep = "Q")
           yq
       }